### 이번 실습에서는 다음 내용들을 배웁니다.


- pandas에서 대용량 데이터를 불러올 때 사용할 수 있는 방법들을 배워봅니다.


- 대용량 데이터 분석에서 주의해야할 점들을 다룹니다.

데이터 링크 : [다운로드](https://drive.google.com/file/d/10Am_lQnYbUhACh_z5HKlN-lBRPgoUAFp/view?usp=share_link)

In [1]:
import pandas as pd
from time import time

### Load Transaction data

- pandas로 데이터를 읽어온 뒤, 기본적인 데이터를 확인해봅니다.

In [2]:
# Load data (pandas version)
start = time()
train = pd.read_csv('./data/opinet/transactions_train.csv')
end = time()

print(f"Loading Time : {end-start} sec.")

Loading Time : 19.239510536193848 sec.


In [6]:
# Check memory usage
mem_usage = train.memory_usage(deep=True).sum() / 1024 / 1024 / 1024
print(f"Memory Usage : {mem_usage:.4} GiB")

Memory Usage : 5.803 GiB


### 데이터를 가져오는 첫번째 방법. parquet

- 훨씬 더 데이터를 빠르게 가져올 수 있는 구조인 parquet 파일로 변환합니다.

In [8]:
# parquet로 다시 만들어봅시다.
start_time = time()
part = pd.read_csv('./data/opinet/transactions_train.csv', 
                   nrows=1000)
stop_time = time()
print(f"Loading Time : {stop_time-start_time} sec.")


Loading Time : 0.0039861202239990234 sec.


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
995,2018-09-20,05943a58bd172641b80919a9bdf14012df940800bc74d0...,661794001,0.152525,2
996,2018-09-20,05943a58bd172641b80919a9bdf14012df940800bc74d0...,661794001,0.152525,2
997,2018-09-20,05943a58bd172641b80919a9bdf14012df940800bc74d0...,661794002,0.152525,2
998,2018-09-20,05943a58bd172641b80919a9bdf14012df940800bc74d0...,661794002,0.152525,2


In [9]:
start_time = time()
part2 = pd.read_csv('./data/opinet/transactions_train.csv',
                    usecols=['t_dat', 'sales_channel_id'])
stop_time = time()

print(f"Loading Time : {stop_time-start_time} sec.")

Loading Time : 12.414984226226807 sec.


In [10]:
mem_usage = part2.memory_usage(deep=True).sum() / 1024 / 1024 / 1024
print(f"Memory Usage : {mem_usage:.4} GiB")

Memory Usage : 1.984 GiB


### 데이터를 가져오는 두번째 방법. I/O

- 데이터를 쪼개서 들고와봅시다.

In [14]:
sales = part["sales_channel_id"].value_counts() * 0
for chunk in pd.read_csv('./data/opinet/transactions_train.csv',
                         chunksize=3000000):
    print(chunk["sales_channel_id"].value_counts())
    sales = sales + chunk["sales_channel_id"].value_counts()

sales

sales_channel_id
2    2076210
1     923790
Name: count, dtype: int64
sales_channel_id
2    2065878
1     934122
Name: count, dtype: int64
sales_channel_id
2    2102438
1     897562
Name: count, dtype: int64
sales_channel_id
2    2076682
1     923318
Name: count, dtype: int64
sales_channel_id
2    2046399
1     953601
Name: count, dtype: int64
sales_channel_id
2    2135860
1     864140
Name: count, dtype: int64
sales_channel_id
2    2003254
1     996746
Name: count, dtype: int64
sales_channel_id
2    2264959
1     735041
Name: count, dtype: int64
sales_channel_id
2    2470145
1     529855
Name: count, dtype: int64
sales_channel_id
2    1929174
1    1070826
Name: count, dtype: int64
sales_channel_id
2    1208863
1     579461
Name: count, dtype: int64


sales_channel_id
2    22379862
1     9408462
Name: count, dtype: int64

### 데이터를 가져오는 세번째 방법. 사용할 데이터만 가져오기

- column을 지정해서 들고와 볼까요?

In [18]:
train_20200601 = train[train['t_dat'] > '2020-06-01']
train_20200601.to_csv('transactions_202006.csv', index=False)